<a href="https://colab.research.google.com/github/OmdenaAI/berlin-germany-estimating-co2/blob/scraping/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

!apt install chromium-chromedriver

!pip install pandas

In [ ]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get("https://ig.ft.com/carbon-food-labelling/")

time.sleep(2)

food = []

try:
  add_buttons = driver.find_elements(By.CSS_SELECTOR, ".scales-item__add");
  add_buttons[1].click()

  items = driver.find_elements(By.CSS_SELECTOR, ".overlay-results .overlay-items .overlay-item")

  if len(items) == 0:
    print("Request Failed!")
    driver.close()
    exit

  count = 1

  for item in items:
    count = count + 1
    if not item.is_displayed():
      overlays = driver.find_elements(By.CSS_SELECTOR, ".overlay-results")
      driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + arguments[1]", overlays[0], 50)

      if not item.is_displayed():
        break

    entry = {}
    entry["item"] = item.find_elements(By.TAG_NAME, "span")[1].text

    item.click()

    show = driver.find_element(By.CSS_SELECTOR, ".scales-show-breakdown")
    showArrow = show.find_elements(By.CSS_SELECTOR, ".show-span") 
    if len(showArrow) > 0:
      show.click()

    table = driver.find_element(By.CSS_SELECTOR, ".breakdown-table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    for row in rows:
      columns = row.find_elements(By.TAG_NAME, "td")

      service = columns[1].text
      if service != "" and service  != "Other":
        entry[service] = columns[2].text
        
    print(entry)
    food.append(entry)
    
    close = driver.find_element(By.CSS_SELECTOR, ".scales-item.scales-item--chosen .scales-item__switch")
    close.click()
  
  driver.close()
except TypeError:
  
  print(food)

#df = pd.DataFrame(food)
#df.to_csv('/content/sample_data/output.csv', index = False);